# Проект по А/B-тестированию

## Подготовка данных

In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

In [2]:
# Загрузка данных
marketing_events = pd.read_csv('/datasets/ab_project_marketing_events.csv')
new_users = pd.read_csv('/datasets/final_ab_new_users.csv')
events = pd.read_csv('/datasets/final_ab_events.csv')
participants = pd.read_csv('/datasets/final_ab_participants.csv')

In [3]:
# Описание проекта
print("Проект по A/B-тестированию: Оценка результатов рекомендательной системы")
print("Цель: Проверить корректность проведения A/B-теста и проанализировать его результаты.")
print("Задачи:")
print("- Проверить отсутствие пересечений с конкурирующими тестами и участников в двух группах.")
print("- Проверить равномерность распределения пользователей в группах A и B.")
print("- Оценить, достигнуто ли улучшение каждой метрики на 10% за 14 дней.")
print("- Проанализировать результаты и определить бизнес-перспективы изменений.")
print("\n")

Проект по A/B-тестированию: Оценка результатов рекомендательной системы
Цель: Проверить корректность проведения A/B-теста и проанализировать его результаты.
Задачи:
- Проверить отсутствие пересечений с конкурирующими тестами и участников в двух группах.
- Проверить равномерность распределения пользователей в группах A и B.
- Оценить, достигнуто ли улучшение каждой метрики на 10% за 14 дней.
- Проанализировать результаты и определить бизнес-перспективы изменений.




In [4]:
# Вывод общей информации о каждом датасете
print("Информация о marketing_events:")
print(marketing_events.info())

print("\nИнформация о new_users:")
print(new_users.info())

print("\nИнформация о events:")
print(events.info())

print("\nИнформация о participants:")
print(participants.info())

Информация о marketing_events:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes
None

Информация о new_users:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
None

Информация о events:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count  

In [5]:
# Описание датасетов
print("Датасеты:")
print("- marketing_events: Календарь маркетинговых событий на 2020 год.")
print("- new_users: Пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.")
print("- events: События новых пользователей с 7 декабря 2020 по 4 января 2021 года.")
print("- participants: Таблица участников тестов.")
print("\n")

Датасеты:
- marketing_events: Календарь маркетинговых событий на 2020 год.
- new_users: Пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.
- events: События новых пользователей с 7 декабря 2020 по 4 января 2021 года.
- participants: Таблица участников тестов.




In [6]:
# Выведем образцы данных из каждого датасета для ознакомления
print("Примеры данных:")
print("- Пример данных из marketing_events:")
display(marketing_events.head())
print("\n- Пример данных из new_users:")
display(new_users.head())
print("\n- Пример данных из events:")
display(events.head())
print("\n- Пример данных из participants:")
display(participants.head())

Примеры данных:
- Пример данных из marketing_events:


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11



- Пример данных из new_users:


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone



- Пример данных из events:


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99



- Пример данных из participants:


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


## Предобработка данных

In [7]:
# Удаление лишних пробелов из столбца 'region'
new_users['region'] = new_users['region'].str.strip()

# Удаление лишних пробелов из столбца 'device'
new_users['device'] = new_users['device'].str.strip()

# Проверка количества значений с лишними пробелами после удаления
print("Количество значений с лишними пробелами в столбце 'region' new_users после удаления:",
      new_users['region'].str.len().lt(len('region')).sum())

print("Количество значений с лишними пробелами в столбце 'device' new_users после удаления:",
      new_users['device'].str.len().lt(len('device')).sum())

Количество значений с лишними пробелами в столбце 'region' new_users после удаления: 52578
Количество значений с лишними пробелами в столбце 'device' new_users после удаления: 21683


In [8]:
# Преобразование типов
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

new_users['first_date'] = pd.to_datetime(new_users['first_date'])

events['event_dt'] = pd.to_datetime(events['event_dt'])
events['details'] = events['details'].astype(float)  # Переводим стоимость покупки в числовой формат

In [9]:
# Проверка на пропущенные значения
print("Пропущенные значения в marketing_events:")
print(marketing_events.isnull().sum())

print("\nПропущенные значения в new_users:")
print(new_users.isnull().sum())

print("\nПропущенные значения в events:")
print(events.isnull().sum())

print("\nПропущенные значения в participants:")
print(participants.isnull().sum())

Пропущенные значения в marketing_events:
name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

Пропущенные значения в new_users:
user_id       0
first_date    0
region        0
device        0
dtype: int64

Пропущенные значения в events:
user_id            0
event_dt           0
event_name         0
details       377577
dtype: int64

Пропущенные значения в participants:
user_id    0
group      0
ab_test    0
dtype: int64


In [10]:
# Проверка на дубликаты
print("\nКоличество дубликатов в marketing_events:", marketing_events.duplicated().sum())
print("Количество дубликатов в new_users:", new_users.duplicated().sum())
print("Количество дубликатов в events:", events.duplicated().sum())
print("Количество дубликатов в participants:", participants.duplicated().sum())


Количество дубликатов в marketing_events: 0
Количество дубликатов в new_users: 0
Количество дубликатов в events: 0
Количество дубликатов в participants: 0


In [11]:
print("Информация о marketing_events:")
print(marketing_events.info())
print("\nИнформация о new_users:")
print(new_users.info())
print("\nИнформация о events:")
print(events.info())
print("\nИнформация о participants:")
print(participants.info())

Информация о marketing_events:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes
None

Информация о new_users:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB
None

Информация о events:
<class 'p

In [12]:
# Список всех уникальных тестов
all_tests = participants['ab_test'].unique()

# Находим тесты, отличные от исследуемого A/B-теста
other_tests = [test for test in all_tests if test != 'recommender_system_test']

print("Сторонние тесты:", other_tests)

Сторонние тесты: ['interface_eu_test']


In [19]:
start_date = pd.to_datetime('2020-12-07')
end_date = pd.to_datetime('2021-01-04')

# Получение списка пользователей стороннего теста
external_test_users = participants[participants['ab_test'] == 'interface_eu_test']['user_id']

# Фильтрация пользователей текущего теста
filtered_new_users = new_users[
    (new_users['first_date'] >= start_date) & 
    (new_users['first_date'] <= end_date) &
    (~new_users['user_id'].isin(external_test_users)) #фильтруем пользователей, чьи идентификаторы НЕ содержатся в списке external_test_users,
]

# Фильтрация событий текущего теста
filtered_events = events[
    (events['event_dt'] >= start_date) & 
    (events['event_dt'] <= end_date) &
    (events['user_id'].isin(filtered_new_users['user_id']))
]

print("Количество пользователей после фильтрации:", len(filtered_new_users))
print("Количество событий после фильтрации:", len(filtered_events))

Количество пользователей после фильтрации: 50166
Количество событий после фильтрации: 358390


## Проверка корректности проведения теста

In [ ]:
# Определение переменных из технического задания
expected_start_date = pd.to_datetime('2020-12-07')
expected_end_date = pd.to_datetime('2021-01-04')
expected_audience = 0.15  # 15% новых пользователей из региона EU
expected_users = 6000

# Проверка стартовой и конечной даты регистрации пользователей
if (filtered_new_users['first_date'].min() >= expected_start_date) and (filtered_new_users['first_date'].max() <= expected_end_date):
    print("Дата регистрации пользователей соответствует требованиям.")
else:
    print("Дата регистрации пользователей не соответствует требованиям.")

# Проверка аудитории
expected_audience_size = len(new_users) * expected_audience
actual_audience_size = len(filtered_new_users)
if abs(expected_audience_size - actual_audience_size) < 1:
    print("Аудитория теста соответствует требованиям.")
else:
    print("Аудитория теста не соответствует требованиям.")

# Проверка ожидаемого количества участников
if len(participants) == expected_users:
    print("Ожидаемое количество участников соответствует требованиям.")
else:
    print("Ожидаемое количество участников не соответствует требованиям.")


In [ ]:
# Проверка пересечения с маркетинговыми событиями
intersects_with_marketing = marketing_events[
    (marketing_events['start_dt'] <= expected_end_date) &
    (marketing_events['finish_dt'] >= expected_start_date)
]

if intersects_with_marketing.empty:
    print("Тест не пересекается с маркетинговыми событиями.")
else:
    display("Тест пересекается с маркетинговыми событиями:", intersects_with_marketing)

Исходя из вышеуказанных результатов проверок, можно сделать следующие выводы:

- Дата регистрации пользователей соответствует требованиям.

- Аудитория теста: Размер аудитории теста не соответствует ожидаемому. Возможно, ошибка в фильтрации новых пользователей или другие проблемы. Требуется пересмотреть и скорректировать этот аспект.

- Ожидаемое количество участников: Количество участников не соответствует ожидаемому значению. Возможно, при формировании списка участников произошла ошибка или недоразумение. Следует повторно проверить и скорректировать этот аспект.

- Пересечение с маркетинговыми событиями: Тест пересекается с несколькими маркетинговыми событиями, включая рождественскую акцию и лотерею. Это может повлиять на результаты теста, так как пользователи могли быть также затронуты другими активностями. Необходимо оценить влияние этих событий на результаты теста.

В целом, на данном этапе можно сказать, что проведение теста имеет некоторые недоразумения и несоответствия требованиям.

## Исследовательский анализ данных.

- Количество событий на пользователя одинаково распределены в выборках?
- Как число событий в выборках распределено по дням?
- Как меняется конверсия в воронке в выборках на разных этапах?
- Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

In [ ]:
# Объединение данных пользователей и участников теста
merged_data = filtered_events.merge(participants, on='user_id', how='inner')

# Вычисление среднего количества событий на пользователя для каждой группы
events_per_user_control = merged_data[merged_data['group'] == 'A'].groupby('user_id')['event_name'].count()
events_per_user_variant = merged_data[merged_data['group'] == 'B'].groupby('user_id')['event_name'].count()

mean_events_control = events_per_user_control.mean()
mean_events_variant = events_per_user_variant.mean()

print("Среднее количество событий на пользователя в контрольной группе:", mean_events_control)
print("Среднее количество событий на пользователя в вариантной группе:", mean_events_variant)

Из проведенного анализа среднего количества событий на пользователя в контрольной и вариантной группах можно сделать следующие выводы:

В контрольной группе среднее количество событий на пользователя составляет приблизительно 5.72. В то же время, в вариантной группе это значение равно примерно 5.60. Таким образом, наблюдается небольшая разница в среднем количестве событий между двумя группами.

In [ ]:
# Вычисление числа событий по дням для каждой группы
events_per_day_control = merged_data[merged_data['group'] == 'A'].groupby(merged_data['event_dt'].dt.date)['event_name'].count()
events_per_day_variant = merged_data[merged_data['group'] == 'B'].groupby(merged_data['event_dt'].dt.date)['event_name'].count()

# Вывод сводной статистики для числа событий в контрольной группе
print("Сводная статистика для числа событий в контрольной группе:")
print(events_per_day_control.describe())

# Вывод сводной статистики для числа событий в вариантной группе
print("\nСводная статистика для числа событий в вариантной группе:")
print(events_per_day_variant.describe())

# Построение графиков распределения событий по дням для каждой группы
plt.figure(figsize=(12, 6))
plt.plot(events_per_day_control.index, events_per_day_control.values, label='Контрольная группа')
plt.plot(events_per_day_variant.index, events_per_day_variant.values, label='Вариантная группа')
plt.xlabel('Дата')
plt.ylabel('Число событий')
plt.title('Распределение числа событий по дням')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Исходя из сводной статистики для числа событий в контрольной и вариантной группах, можно сделать следующие выводы:

Контрольная группа:

- Количество наблюдений (событий): 14
- Среднее количество событий на пользователя: около 2636
- Стандартное отклонение: приблизительно 973
- Минимальное количество событий на пользователя: 1375
- Медиана (50-й перцентиль): 2757
- 25-й перцентиль: 1646.5
- 75-й перцентиль: 3463.5
- Максимальное количество событий на пользователя: 4040

Вариантная группа:

- Количество наблюдений (событий): 14
- Среднее количество событий на пользователя: около 2018
- Стандартное отклонение: приблизительно 463
- Минимальное количество событий на пользователя: 1402
- Медиана (50-й перцентиль): 2217
- 25-й перцентиль: 1523
- 75-й перцентиль: 2343
- Максимальное количество событий на пользователя: 2636

Сравнивая сводные статистики для обеих групп, видно, что среднее количество событий на пользователя в контрольной группе выше, чем в вариантной группе. Стандартное отклонение также значительно выше в контрольной группе, что может указывать на большую вариативность данных.

In [ ]:
# Вычисление количества пользователей на каждом этапе воронки для каждой группы
funnel_control = merged_data[merged_data['group'] == 'A'].groupby('event_name')['user_id'].nunique()
funnel_variant = merged_data[merged_data['group'] == 'B'].groupby('event_name')['user_id'].nunique()

# Вычисление конверсии для каждого этапа воронки для каждой группы
conversion_control = funnel_control / funnel_control.iloc[0]
conversion_variant = funnel_variant / funnel_variant.iloc[0]

# Вывод сводной статистики
print("Сводная статистика для числа пользователей на каждом этапе воронки (контрольная группа):\n", funnel_control)
print("\nСводная статистика для числа пользователей на каждом этапе воронки (вариантная группа):\n", funnel_variant)

# Вывод конверсии
print("\nКонверсия в воронке (контрольная группа):\n", conversion_control)
print("\nКонверсия в воронке (вариантная группа):\n", conversion_variant)

# Построение графиков конверсии в воронке для каждой группы
plt.figure(figsize=(10, 6))
plt.plot(conversion_control.index, conversion_control.values, label='Контрольная группа')
plt.plot(conversion_variant.index, conversion_variant.values, label='Вариантная группа')
plt.xlabel('Этап воронки')
plt.ylabel('Конверсия')
plt.title('Конверсия в воронке')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Исходя из проведенного анализа, мы получили следующие результаты:

Сводная статистика для числа пользователей на каждом этапе воронки показывает, что в контрольной группе:

- Количество пользователей на этапе "login" составляет 6453.
- Количество пользователей на этапе "product_cart" составляет 2057.
- Количество пользователей на этапе "product_page" составляет 4248.
- Количество пользователей на этапе "purchase" составляет 2194.

В вариантной группе:

- Количество пользователей на этапе "login" составляет 5043.
- Количество пользователей на этапе "product_cart" составляет 1645.
- Количество пользователей на этапе "product_page" составляет 3217.
- Количество пользователей на этапе "purchase" составляет 1621.

Анализ конверсии в воронке позволяет нам сделать следующие выводы:

- На первом этапе "login" конверсия составляет 100% для обеих групп, так как это стартовый этап.
- На втором этапе "product_cart" конверсия в контрольной группе составляет примерно 31.88%, а в вариантной группе - около 32.62%.
- На третьем этапе "product_page" конверсия в контрольной группе составляет примерно 65.83%, а в вариантной группе - около 63.79%.
- На последнем этапе "purchase" конверсия в контрольной группе составляет примерно 34.00%, а в вариантной группе - около 32.14%.

Исходя из анализа, мы видим, что на каждом последующем этапе воронки конверсия немного снижается как в контрольной, так и в вариантной группе. При этом, на всех этапах воронки конверсия в контрольной группе выше, чем в вариантной группе, хотя разница не слишком значительна.

**Для успешного A/B-тестирования важно учесть следующие особенности данных:**

- Равномерное распределение пользователей по тестовым группам для более точного сравнения результатов.
- Исключение пересечений с другими тестами и активностями, чтобы избежать искажений результатов.
- Анализ статистической значимости результатов A/B-теста с помощью статистических тестов.
- Оценка практической значимости изменений и их влияния на ключевые метрики бизнеса.
- Учет сезонных, маркетинговых и других внешних факторов, которые могут повлиять на результаты теста.

## Оценка результатов A/B-тестирования

- Что можно сказать про результаты A/B-тестирования?
- Проверьте статистическую разницу долей z-критерием.

Для оценки результатов A/B-тестирования нам необходимо провести статистическую оценку различий между двумя группами с помощью z-критерия для разности долей. Этот анализ позволит определить, являются ли различия между контрольной и вариантной группами статистически значимыми.

Для начала вспомним доли конверсий на каждом этапе воронки в обеих группах:

Контрольная группа:

- Доля конверсии на этапе "product_cart" в контрольной группе: 0.318766
- Доля конверсии на этапе "product_page" в контрольной группе: 0.658298
- Доля конверсии на этапе "purchase" в контрольной группе: 0.339997

Вариантная группа:

- Доля конверсии на этапе "product_cart" в вариантной группе: 0.326195
- Доля конверсии на этапе "product_page" в вариантной группе: 0.637914
- Доля конверсии на этапе "purchase" в вариантной группе: 0.321436

**Теперь проведем статистическую оценку разницы долей между группами с помощью z-критерия для разности долей.**

In [17]:
# Количество пользователей и конверсий на каждом этапе воронки в контрольной группе
control_users = [6453, 2057, 4248, 2194]
control_conversions = [1.0, 0.318766, 0.658298, 0.339997]

# Количество пользователей и конверсий на каждом этапе воронки в вариантной группе
variant_users = [5043, 1645, 3217, 1621]
variant_conversions = [1.0, 0.326195, 0.637914, 0.321436]

alpha = 0.05  # Уровень значимости

# Проведение z-теста для каждого этапа воронки
for i in range(len(control_users)):
    print(f"Этап {i+1}:")
    z_stat, p_value = proportions_ztest([control_conversions[i] * control_users[i], variant_conversions[i] * variant_users[i]], [control_users[i], variant_users[i]])
    p_values = [p_value]  # Список p-значений для множественной коррекции
    reject, p_corrected, _, _ = multipletests(p_values, alpha=alpha, method='bonferroni')
    adjusted_alpha = alpha / len(control_users)  # Применение поправки Бонферрони
    print(f"P-значение: {p_corrected[0]}")
    if p_corrected[0] < adjusted_alpha:
        print("Статистически значимая разница")
    else:
        print("Статистически незначимая разница")
    print()

Этап 1:
P-значение: nan
Статистически незначимая разница

Этап 2:
P-значение: 0.6307520860255911
Статистически незначимая разница

Этап 3:
P-значение: 0.06755814558825712
Статистически незначимая разница

Этап 4:
P-значение: 0.22886408207312514
Статистически незначимая разница



Полученные результаты говорят о том, что на всех этапах воронки разница между контрольной и вариантной группами не является статистически значимой. Это означает, что на основе проведенного анализа данных нельзя сделать вывод о том, что изменения в продукте (в данном случае рекомендательной системе) привели к статистически значимому увеличению или уменьшению конверсии на каком-либо этапе воронки.

## Выводы

**Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования. Сделайте общее заключение о корректности проведения теста.**

Итак, проведенный исследовательский анализ данных и оценка результатов A/B-тестирования позволяют сделать следующие выводы:

**Исследовательский анализ данных:**

- Дата регистрации пользователей соответствует требованиям.

- Аудитория теста: Размер аудитории теста не соответствует ожидаемому. Это может быть следствием ошибок в фильтрации или других факторов. Требуется дополнительный анализ и корректировка для обеспечения корректности тестирования.

- Ожидаемое количество участников: Наблюдается расхождение между ожидаемым и реальным количеством участников. Это может быть связано с ошибками при формировании списка участников. Необходимо тщательно пересмотреть этот аспект.

- Пересечение с маркетинговыми событиями: Тест пересекается с другими маркетинговыми событиями, такими как рождественская акция и лотерея. Это может повлиять на результаты теста из-за воздействия других активностей на пользователей. Важно оценить, как это влияние может сказаться на результатах.

**Оценка результатов A/B-тестирования:**

В результате оценки результатов A/B-тестирования было получено следующее:

- На каждом этапе воронки наблюдается незначительное снижение конверсии в вариантной группе по сравнению с контрольной. Однако эти различия не являются статистически значимыми, что означает, что изменения в рекомендательной системе не оказали значительного влияния на конверсию.

- Сравнение сводных статистик показало, что количество событий на пользователя немного выше в контрольной группе. При этом стандартное отклонение также выше, что указывает на большую вариативность данных в контрольной группе.

**Общее заключение:**

На основе вышеизложенного, можно сделать вывод о том, что проведение теста имеет некоторые несоответствия требованиям и недоразумения, которые могли повлиять на результаты. Однако статистические проверки и анализ показали, что различия между контрольной и вариантной группами не являются статистически значимыми. Это означает, что изменения в рекомендательной системе, которые были внесены, скорее всего, не оказали существенного влияния на поведение пользователей.

В целом, для более точной и объективной оценки результатов A/B-тестирования необходимо более внимательное внимание к деталям формирования групп и аудитории теста, а также проведение дополнительных анализов для исключения внешних факторов, которые могли повлиять на результаты.